# Amazon Personalize: la utilización del texto como metadatos de elementos no estructurados

La relevancia de las recomendaciones que ofrece con Amazon Personalize depende de los datos disponibles al momento de generar estas recomendaciones. Amazon Personalize utiliza las interacciones históricas de los usuarios, los atributos de los elementos y los metadatos de los usuarios para aprender qué elementos son más relevantes para cada usuario. Los datos principales que requiere Amazon Personalize son las interacciones entre los usuarios y los elementos. Las interacciones que los usuarios tengan con los elementos de un catálogo, como hacer clic en un producto, leer un artículo, mirar un video o comprar un producto, son una señal importante de qué han encontrado relevante en el pasado. La acción de incluir elementos y atributos de los usuarios, también conocidos como metadatos, puede incrementar la relevancia de las recomendaciones, en especial para elementos nuevos que son similares a los que los usuarios han encontrado relevantes. Sin embargo, los metadatos estructurados, como la categoría de un elemento, el estilo o el género, pueden no siempre estar disponibles de inmediato ni proveer toda la información que se tiene en las descripciones narrativas. Ahora, Amazon Personalize permite agregar metadatos no estructurados, como descripciones de productos, transcripciones de videos o textos en los artículos, con los otros atributos de los elementos. Amazon Personalize actúa como alojamiento, administra y utiliza modelos de procesamiento de lenguajes naturales (NLP) de forma automática para procesar textos y utilizarlos para mejorar el rendimiento de las soluciones de Amazon Personalize.

Este cuaderno demostrará cómo el texto utilizado como descripciones de productos puede incluirse como metadatos de elementos no estructurados para mejorar la relevancia de las recomendaciones.

Los datos de Amazon Reviews de la categoría de Amazon Prime Pantry se utilizan para los conjuntos de datos de interacciones y elementos.

Al momento de considerar la opción de incluir texto en los conjuntos de datos de los elementos, tenga en cuenta las siguientes prácticas recomendadas.
- Se prefiere el texto que ha sido editado para ser conciso, relevante e informativo para cada elemento, donde los detalles más pertinentes se mencionan en primer lugar dentro del texto, por sobre el contenido generado por los usuarios que puede ser menos relevante o consistente.
- Una columna de texto poblada de manera escasa disminuirá el impacto positivo de la inclusión de texto en los elementos del conjunto de datos.
- Limpie todo el texto de marcados y formatos de espacios en blanco superfluos antes de adjuntarlo a los elementos del conjunto de datos.
- Por el momento, solo el idioma inglés es compatible con el campo de texto.
- Actualmente, los campos de texto solo se consideran para las recetas de User-Personalization y Personalized-Ranking.

Se crearán dos grupos de conjuntos de datos que incluirán o no datos con descripciones de los elementos, para probar los modelos por separado y, luego, comparar sus resultados en línea y fuera de línea.

In [1]:
import pandas as pd
import json
import numpy as np
from datetime import datetime
import boto3
import time
from time import sleep
from lxml import html

## Cargar y revisar los conjuntos de datos

Comenzaremos cargando el conjunto de datos de las reseñas de Prime Pantry. Deberá completar el formulario para acceder a los archivos de datos:

http://deepyeti.ucsd.edu/jianmo/amazon/index.html

Cita:
> Justifying recommendations using distantly-labeled reviews and fined-grained aspects (Justificar las recomendaciones mediante el uso de reseñas etiquetadas de forma lejana y aspectos detallados)  
> Jianmo Ni, Jiacheng Li, Julian McAuley  
> Empirical Methods in Natural Language Processing (EMNLP), 2019 (Métodos empíricos en el procesamiento de lenguajes naturales [2019]) [pdf](http://cseweb.ucsd.edu/~jmcauley/pdfs/emnlp19a.pdf)

In [2]:
data_dir = 'raw_data'
!mkdir $data_dir

!cd $data_dir && \
    wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Prime_Pantry.json.gz && \
    wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Prime_Pantry.json.gz

mkdir: cannot create directory ‘raw_data’: File exists
--2021-07-13 22:06:52--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Prime_Pantry.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45435146 (43M) [application/octet-stream]
Saving to: ‘Prime_Pantry.json.gz’

Prime_Pantry.json.g 100%[===================>]  43.33M  12.3MB/s    in 4.8s    

2021-07-13 22:06:57 (9.01 MB/s) - ‘Prime_Pantry.json.gz’ saved [45435146/45435146]

--2021-07-13 22:06:57--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Prime_Pantry.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5281662 (5.0M) [application/octet-stream]
Saving to: ‘meta_Prime_Pantry.json.gz’

meta_Prime_Pa

### Cargar y revisar los datos de reseñas

Comenzaremos con la carga del conjunto de datos de las reseñas para los productos de Prime Pantry y la ejecución de algunos comandos para ver con qué estamos trabajando.

In [3]:
pantry_df = pd.read_json(data_dir + '/Prime_Pantry.json.gz', lines=True, compression='infer')
pantry_df.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,image,style
0,5,True,"12 14, 2014",A1NKJW0TNRVS7O,B0000DIWNZ,Tamara M.,Good clinging,Clings well,1418515200,NaN,NaN,NaN
1,4,True,"11 20, 2014",A2L6X37E8TFTCC,B0000DIWNZ,Amazon Customer,Fantastic buy and a good plastic wrap. Even t...,Saran could use more Plus to Cling better.,1416441600,NaN,NaN,NaN
2,4,True,"10 11, 2014",A2WPR4W6V48121,B0000DIWNZ,noname,ok,Four Stars,1412985600,NaN,NaN,NaN
3,3,False,"09 1, 2014",A27EE7X7L29UMU,B0000DIWNZ,ZapNZs,Saran Cling Plus is kind of like most of the C...,"The wrap is fantastic, but the dispensing, cut...",1409529600,4,NaN,NaN
4,4,True,"08 10, 2014",A1OWT4YZGB5GV9,B0000DIWNZ,Amy Rogers,This is my go to plastic wrap so there isn't m...,has been doing it's job for years,1407628800,NaN,NaN,NaN


In [4]:
pantry_df.shape

(471614, 12)

¿Qué podemos aprender de este resultado? Hay más de 471 000 reseñas y 12 columnas de datos. La columna `asin` es el identificador único de elementos, la columna `reviewerID` es el identificador único de usuario, la columna `unixReviewTime` es la marca temporal de la reseña y la columna `overall` indica la positividad de la reseña en una escala del 1 al 5. Utilizaremos este archivo como la base para el conjunto de datos de las interacciones para Personalize. 

### Crear y guardar el conjunto de datos de las interacciones

Crearemos el conjunto de datos de las interacciones mediante la reducción de las filas que queremos incluir. El primer paso es aislar solo las reseñas positivas. Para esto, supondremos que cualquier reseña con una calificación general de 4 o más constituye una reseña positiva. Cualquier reseña de 3 o menos se considera mediocre o negativa.

In [5]:
positive_reviews_df = pantry_df[pantry_df['overall'] > 3]
positive_reviews_df.shape

(387692, 12)

Contamos, entonces, con 387 000 reseñas positivas. Son suficientes para un modelo de prueba de Personalize.

A continuación, reduciremos el conjunto de datos a las columnas que necesitamos y agregaremos una columna de `EVENT_TYPE` para indicar los tipos de eventos que queremos capturar. La adición de una columna `EVENT_TYPE` simplificará la exploración de pruebas de eventos en tiempo real más adelante, si elige hacerla (ya que `eventType` es un campo requerido por la API [PutEvents](https://docs.aws.amazon.com/personalize/latest/dg/API_UBS_PutEvents.html)).

In [6]:
positive_reviews_df = positive_reviews_df[['reviewerID', 'asin', 'unixReviewTime', 'overall']]
positive_reviews_df['EVENT_TYPE']='reviewed'

positive_reviews_df.head()

,reviewerID,asin,unixReviewTime,overall,EVENT_TYPE
0,A1NKJW0TNRVS7O,B0000DIWNZ,1418515200,5,reviewed
1,A2L6X37E8TFTCC,B0000DIWNZ,1416441600,4,reviewed
2,A2WPR4W6V48121,B0000DIWNZ,1412985600,4,reviewed
4,A1OWT4YZGB5GV9,B0000DIWNZ,1407628800,4,reviewed
5,A1GN2ADKF1IE7K,B0000DIWNZ,1405296000,5,reviewed


Por último, compruebe el estado del valor de la columna `unixReviewTime`. Ya que Personalize crea modelos de secuencias basados en la fecha y la hora de cada interacción, es importante que estas marcas temporales estén representadas en el formato esperado en cada interacción, para que sean interpretadas de forma correcta.

Seleccione un valor para la columna `unixReviewTime` y analícelo en un formato de fecha legible para humanos para verificar que sea razonable.

In [7]:
time_stamp = positive_reviews_df.iloc[50]['unixReviewTime']
print(time_stamp)
print(datetime.utcfromtimestamp(time_stamp).strftime('%Y-%m-%d %H:%M:%S'))

1321488000
2011-11-17 00:00:00


El valor de las marcas temporales se ve bien. Vamos a obtener un resumen final de la información de nuestro conjunto de datos.

In [8]:
positive_reviews_df.describe(include='all')

,reviewerID,asin,unixReviewTime,overall,EVENT_TYPE
count,387692,387692,3.876920e+05,387692.000000,387692
unique,202254,10584,NaN,NaN,1
top,A35Q0RBM3YNQNF,B00XA9DADC,NaN,NaN,reviewed
freq,176,5288,NaN,NaN,387692
mean,NaN,NaN,1.468847e+09,4.847227,NaN
std,NaN,NaN,4.314975e+07,0.359769,NaN
min,NaN,NaN,1.073693e+09,4.000000,NaN
25%,NaN,NaN,1.447200e+09,5.000000,NaN
50%,NaN,NaN,1.474718e+09,5.000000,NaN
75%,NaN,NaN,1.498435e+09,5.000000,NaN


Tenemos 387 000 reseñas para 202 000 usuarios/reseñadores diferentes en 10 000 productos únicos. Esta es la base del conjunto de datos de interacciones.

Antes de utilizar estos datos como el conjunto de datos de interacciones, necesita volver a nombrar las columnas para que correspondan con las que espera Personalize.

In [9]:
positive_reviews_df.rename(columns = {'reviewerID':'USER_ID', 'asin':'ITEM_ID', 
                              'unixReviewTime':'TIMESTAMP', 'overall': 'EVENT_VALUE'}, inplace = True)
positive_reviews_df

,USER_ID,ITEM_ID,TIMESTAMP,EVENT_VALUE,EVENT_TYPE
0,A1NKJW0TNRVS7O,B0000DIWNZ,1418515200,5,reviewed
1,A2L6X37E8TFTCC,B0000DIWNZ,1416441600,4,reviewed
2,A2WPR4W6V48121,B0000DIWNZ,1412985600,4,reviewed
4,A1OWT4YZGB5GV9,B0000DIWNZ,1407628800,4,reviewed
5,A1GN2ADKF1IE7K,B0000DIWNZ,1405296000,5,reviewed
...,...,...,...,...,...
471609,A19GSVHXVT5NNF,B01HI8JVI8,1494892800,5,reviewed
471610,ABSCTKLX9F9IU,B01HI8JVI8,1493769600,5,reviewed
471611,A2R33RCWKDHZ3L,B01HI8JVI8,1492646400,5,reviewed
471612,A2INGHYEXZDHMC,B01HI8JVI8,1492560000,5,reviewed


Por último, guarde el marco de datos de las reseñas positivas como un CSV. Más adelante en este cuaderno, se guardará este CSV en Personalize.

In [10]:
interactions_filename = "interactions.csv"
positive_reviews_df.to_csv(interactions_filename, index=False, float_format='%.0f')

### Cargar y revisar los metadatos de elementos

Una vez que se hayan establecido los conjuntos de datos de las interacciones, trabajaremos con los conjuntos de datos de los elementos. Aquí encontrará el valor de texto no estructurado que se incluirá en el modelo.

Al igual que el conjunto de datos de las reseñas, el archivo de metadatos de los elementos de Prime Pantry también se representa en JSON. Dada la naturaleza anidada de este archivo, esto presentará algunos desafíos para formatear los datos como los necesitamos.

Comenzaremos por cargar el archivo de metadatos en un marco de datos y observar los datos.

In [11]:
pantry_meta_df = pd.read_json('raw_data/meta_Prime_Pantry.json.gz', lines=True, compression='infer')
pantry_meta_df

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Sink your sweet tooth into MILK DUDS Candya d...,,"HERSHEY'S Milk Duds Candy, 5 Ounce(Halloween C...","[B019KE37WO, B007NQSWEU]",,Milk Duds,[],[],[],"{'ASIN: ': 'B00005BPJO', 'Item model number:':...","<img src=""https://m.media-amazon.com/images/G/...",,NaT,$5.00,B00005BPJO,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
1,[],,[Sink your sweet tooth into MILK DUDS Candya d...,,"HERSHEY'S Milk Duds Candy, 5 Ounce(Halloween C...","[B019KE37WO, B007NQSWEU]",,Milk Duds,[],[],[],"{'ASIN: ': 'B00005BPJO', 'Item model number:':...","<img src=""https://m.media-amazon.com/images/G/...",,NaT,$5.00,B00005BPJO,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
2,[],,[A perfect Lentil soup starts with Goya Lentil...,,"Goya Dry Lentils, 16 oz","[B003SI144W, B000VDRKEK]",,Goya,[],[],"[B074MFVZG7, B079PTH69L, B000VDRKEK, B074M9T81...",{'ASIN: ': 'B0000DIF38'},"<img src=""https://images-na.ssl-images-amazon....",,NaT,,B0000DIF38,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
3,[],,[Saran Premium Wrap is an extra tough yet easy...,,"Saran Premium Plastic Wrap, 100 Sq Ft","[B01MY5FHT6, B000PYF8VM, B000SRMDFA, B07CX6LN8...",,Saran,[],[],"[B077QLSLRQ, B00JPKW1RQ, B000FE2IK6, B00XUJHJ9...",{'Domestic Shipping: ': 'This item can only be...,"<img src=""https://images-na.ssl-images-amazon....",,NaT,,B0000DIWNI,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[200 sq ft (285 ft x 11-3/4 in x 18.6 m2). Eas...,,"Saran Cling Plus Plastic Wrap, 200 Sq Ft",[],,Saran,[],[],[B0014CZ0TE],{'Domestic Shipping: ': 'This item can only be...,"<img src=""https://images-na.ssl-images-amazon....",,NaT,,B0000DIWNZ,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10808,[],,[These bars are where our journey started and ...,,"KIND Bars, Caramel Almond &amp; Sea Salt, Glut...",[],,KIND,[],"26,259 in Grocery & Gourmet Food (","[B00JQQAN60, B00JQQAWSY, B0111K7V54, B0111K8L9...","{'ASIN: ': 'B01HI76312', 'Item model number:':...","<img src=""https://images-na.ssl-images-amazon....",,NaT,$3.98,B01HI76312,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
10809,[],,[These bars are where our journey started and ...,,"KIND Bars, Maple Glazed Pecan &amp; Sea Salt, ...",[],,KIND,[],"16,822 in Grocery & Gourmet Food (","[B0111K97JC, B00JQQAN60, B0111K8L9Y, B01HI7631...",{'ASIN: ': 'B01HI76790'},"<img src=""https://images-na.ssl-images-amazon....",,NaT,$5.81,B01HI76790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
10810,[],,[These bars are where our journey started and ...,,"KIND Bars, Dark Chocolate Almond &amp; Coconut...",[],,KIND,[],"107,057 in Grocery & Gourmet Food (","[B0111K7V54, B01HI76312, B00JQQAL0S, B0111K97J...",{'ASIN: ': 'B01HI76SA8'},"<img src=""https://images-na.ssl-images-amazon....",,NaT,$4.98,B01HI76SA8,[],[]
10811,[],,[These bars are where our journey started and ...,,"KIND Bars, Honey Roasted Nuts &amp; Sea Salt, ...",[],,KIND,[],"24,648 in Grocery & Gourmet Food (","[B00JQQAN60, B0111K7V54, B01HI76312, B0111K97J...",{'ASIN: ': 'B01HI76XS0'},"<img src=""https://images-na.ssl-images-amazon....",,NaT,$5.81,B01HI76XS0,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [12]:
pantry_meta_df.describe()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
count,10813,10813,10813,10813,10813,10813,10813,10813,10813,10813,10813,10813,10813,10813,0,10813,10813,10813,10813
unique,1,1,9409,1,10782,3957,1,1960,763,4828,5940,10786,4,1,0,1482,10812,8940,8940
top,[],,[],,"Infants' Motrin Concentrated Drops, Fever Redu...",[],,L'Oreal Paris,[],[],[],{},"<img src=""https://images-na.ssl-images-amazon....",,NaN,,B00005BPJO,[],[]
freq,10813,10813,98,10813,2,6754,10813,171,9777,5937,4835,24,10621,10813,NaN,4063,2,1781,1781


¿Qué podemos aprender de esta información? Primero, hay más de 10 000 productos representados en el archivo de metadatos. La mayor parte de las columnas no serán de utilidad para Personalize, ya que no son relevante como características (URL de imágenes, `details`, `also_viewed`, `also_buy`, entre otros) o están mayormente en blanco/dispersas (`category`, `fit`, `tech1`, entre otros). La columna `asin` es el identificador único para cada elemento (aunque parece que hay un duplicado) y `brand` y `price` parecen ser útiles. Utilizaremos la columna `description` para texto no estructurado.

Sin embargo, hay que hacer una limpieza y reformateo de los campos que queremos utilizar en nuestros conjuntos de datos de elementos. Por ejemplo, el campo `price` es un valor de moneda formateado (cadena) y no numérico y el campo `description` se cargó como una matriz de cadenas debido a cómo se representaron y analizaron los valores del archivo JSON original. Por último, los valores `description` también contienen marcadores HTML que necesitan removerse.

Comenzaremos por crear un marco de datos con solo las columnas que necesitamos para cada conjunto de datos de elementos.

In [13]:
items_df = pantry_meta_df.copy()
items_df = items_df[['asin', 'brand', 'price', 'description']]
items_df.head(10)

,asin,brand,price,description
0,B00005BPJO,Milk Duds,$5.00,[Sink your sweet tooth into MILK DUDS Candya d...
1,B00005BPJO,Milk Duds,$5.00,[Sink your sweet tooth into MILK DUDS Candya d...
2,B0000DIF38,Goya,,[A perfect Lentil soup starts with Goya Lentil...
3,B0000DIWNI,Saran,,[Saran Premium Wrap is an extra tough yet easy...
4,B0000DIWNZ,Saran,,[200 sq ft (285 ft x 11-3/4 in x 18.6 m2). Eas...
5,B0000GH6UG,Ibarra,,"[Ibarra Chocolate, 19 Oz, , ]"
6,B0000KC2BK,Knorr,$3.09,[Knorr Granulated Chicken Flavor Bouillon is a...
7,B0001E1IN8,Castillo,,[Red chili habanero sauces. They are present t...
8,B00032E8XK,Chicken of the Sea,$1.48,[Chicken of the Sea Solid White Albacore Tuna ...
9,B0005XMTHE,Smucker's,$2.29,"[Helps build muscles with bcaa's amino acids, ..."


Luego, descartaremos las filas duplicadas basándonos en el valor de la columna `asin`. Debería haber solo una duplicada, según el resultado `describe()` anterior.

In [14]:
items_df = items_df.drop_duplicates(subset=['asin'], keep='last')
items_df.shape

(10812, 4)

A continuación, nos enfocaremos en reformatear y limpiar los valores de la columna `description`. Como puede observar arriba, `description` se representa en este momento como una matriz de cadenas (porque es así como se las representa en el archivo JSON). Es necesario comprimir esta matriz en una sola cadena y remover todos los marcadores HTML de cada fragmento.

Para empezar, crearemos dos funciones de utilidad que se emplearán para limpiar `description` (y luego la columna `title` en el conjunto de datos original cuando queramos mostrar títulos para los productos recomendados).

In [15]:
# Strips and cleans a value of HTML markup and whitespace.
def clean_markup(value):
    s = str(value).strip()
    if s != '':
        s = str(html.fromstring(s).text_content())
        s = ' '.join(s.split())
                
    return s.strip()

# Cleans and reformats the description column value for a dataframe row.
def clean_and_reformat_description(row):
    s = ''
    for el in row['description']:
        el = clean_markup(el)
        if el != '':
            s += ' ' + el
                
    return s.strip()

In [16]:
items_df['description'] = items_df.apply(clean_and_reformat_description, axis=1)
items_df

,asin,brand,price,description
1,B00005BPJO,Milk Duds,$5.00,Sink your sweet tooth into MILK DUDS Candya de...
2,B0000DIF38,Goya,,A perfect Lentil soup starts with Goya Lentils...
3,B0000DIWNI,Saran,,Saran Premium Wrap is an extra tough yet easy ...
4,B0000DIWNZ,Saran,,200 sq ft (285 ft x 11-3/4 in x 18.6 m2). Easy...
5,B0000GH6UG,Ibarra,,"Ibarra Chocolate, 19 Oz"
...,...,...,...,...
10808,B01HI76312,KIND,$3.98,These bars are where our journey started and i...
10809,B01HI76790,KIND,$5.81,These bars are where our journey started and i...
10810,B01HI76SA8,KIND,$4.98,These bars are where our journey started and i...
10811,B01HI76XS0,KIND,$5.81,These bars are where our journey started and i...


A continuación, miraremos la columna `price` y cambiaremos su tipo de una cadena a puntos flotantes.

In [17]:
items_df['price'].value_counts()

          4063
$2.99      114
$3.99      113
$4.99      103
$5.99       87
          ... 
$20.42       1
$32.32       1
$1.52        1
$27.89       1
$39.10       1
Name: price, Length: 1482, dtype: int64

A la celda siguiente con convertir precios vacíos/no numéricos a `np.nan`, y a todas las otras, se les removerá el signo de moneda `$`. Esto permitirá la coerción del tipo a un punto flotante.

In [18]:
def convert_price(row):
    v = str(row['price']).strip().replace('$', '')
    if v == '' or not v.lstrip('-').replace('.', '').isdigit():
        return np.nan
    return v

items_df['price'] = items_df.apply(convert_price, axis=1)
items_df

,asin,brand,price,description
1,B00005BPJO,Milk Duds,5.00,Sink your sweet tooth into MILK DUDS Candya de...
2,B0000DIF38,Goya,NaN,A perfect Lentil soup starts with Goya Lentils...
3,B0000DIWNI,Saran,NaN,Saran Premium Wrap is an extra tough yet easy ...
4,B0000DIWNZ,Saran,NaN,200 sq ft (285 ft x 11-3/4 in x 18.6 m2). Easy...
5,B0000GH6UG,Ibarra,NaN,"Ibarra Chocolate, 19 Oz"
...,...,...,...,...
10808,B01HI76312,KIND,3.98,These bars are where our journey started and i...
10809,B01HI76790,KIND,5.81,These bars are where our journey started and i...
10810,B01HI76SA8,KIND,4.98,These bars are where our journey started and i...
10811,B01HI76XS0,KIND,5.81,These bars are where our journey started and i...


In [19]:
items_df['price'].value_counts()

2.99     114
3.99     113
4.99     103
5.99      87
2.98      76
        ... 
39.10      1
1.84       1
22.95      1
12.17      1
11.09      1
Name: price, Length: 1480, dtype: int64

In [20]:
items_df['price'] = items_df['price'].astype(float)

A continuación, volveremos a nombrar las columnas para que correspondan con los nombres y el formato de nombres con mayúsculas que espera Personalize.

In [21]:
items_df.rename(columns = {'asin':'ITEM_ID', 'brand':'BRAND', 
                              'price':'PRICE', 'description': 'DESCRIPTION'}, inplace = True)
items_df.head(10)

,ITEM_ID,BRAND,PRICE,DESCRIPTION
1,B00005BPJO,Milk Duds,5.00,Sink your sweet tooth into MILK DUDS Candya de...
2,B0000DIF38,Goya,NaN,A perfect Lentil soup starts with Goya Lentils...
3,B0000DIWNI,Saran,NaN,Saran Premium Wrap is an extra tough yet easy ...
4,B0000DIWNZ,Saran,NaN,200 sq ft (285 ft x 11-3/4 in x 18.6 m2). Easy...
5,B0000GH6UG,Ibarra,NaN,"Ibarra Chocolate, 19 Oz"
6,B0000KC2BK,Knorr,3.09,Knorr Granulated Chicken Flavor Bouillon is a ...
7,B0001E1IN8,Castillo,NaN,Red chili habanero sauces. They are present to...
8,B00032E8XK,Chicken of the Sea,1.48,Chicken of the Sea Solid White Albacore Tuna i...
9,B0005XMTHE,Smucker's,2.29,"Helps build muscles with bcaa's amino acids, i..."
10,B0005XNE6E,Snapple,1.99,"At Snapple, we believe lifes a peach. Weve bee..."


Crearemos dos CSV de elementos. Uno tendrá la columna de descripción y el otro no. Utilizaremos cada uno de estos para probar los modelos separados con la misma receta y, así, poder comparar las métricas sin conexión y realizar una revisión en línea de las recomendaciones.

In [22]:
items_with_desc_filename = "items-with-desc.csv"
items_df.to_csv(items_with_desc_filename, index=False, float_format='%.2f')

Otro CSV de elementos con la columna de descripción eliminada.

In [23]:
items_without_desc_df = items_df[['ITEM_ID', 'BRAND', 'PRICE']]
items_without_desc_df.head()

,ITEM_ID,BRAND,PRICE
1,B00005BPJO,Milk Duds,5.0
2,B0000DIF38,Goya,NaN
3,B0000DIWNI,Saran,NaN
4,B0000DIWNZ,Saran,NaN
5,B0000GH6UG,Ibarra,NaN


In [24]:
items_without_desc_filename = "items-without-desc.csv"
items_without_desc_df.to_csv(items_without_desc_filename, index=False, float_format='%.2f')

## Crear grupos de conjuntos de datos y carga de conjuntos de datos

Una vez creados los conjuntos de datos necesarios, es hora de subirlos a Personalize mediante trabajos de importación de conjuntos de datos. Antes de cargar los CSV, necesitamos crear grupos de conjuntos de datos para poder contener nuestros dos enfoques de conjuntos de datos (con descripciones y sin ellas), crear esquemas para nuestros conjuntos de datos y crear conjuntos de datos.

Comenzaremos con la creación de un cliente SDK, cuya función será la de interactuar con Personalize.

In [25]:
personalize = boto3.client('personalize')

### Crear grupos de conjuntos de datos

Creemos nuestros dos grupos de conjuntos de datos.

In [26]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "amazon-pantry-without-desc"
)

dataset_group_without_desc_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:us-east-1:224124347618:dataset-group/amazon-pantry-without-desc",
  "ResponseMetadata": {
    "RequestId": "20bd153c-ebd0-432d-9ef3-522a0d2fd8d4",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:08:18 GMT",
      "x-amzn-requestid": "20bd153c-ebd0-432d-9ef3-522a0d2fd8d4",
      "content-length": "105",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [27]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "amazon-pantry-with-desc"
)

dataset_group_with_desc_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:us-east-1:224124347618:dataset-group/amazon-pantry-with-desc",
  "ResponseMetadata": {
    "RequestId": "9cec53c8-28a3-40e5-bf0e-6f87a03113f7",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:08:18 GMT",
      "x-amzn-requestid": "9cec53c8-28a3-40e5-bf0e-6f87a03113f7",
      "content-length": "102",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Los grupos de conjuntos de datos pueden demorar algunos segundos hasta estar completamente creados. Esperaremos hasta que ambos muestren ACTIVE (Activo) como estado.

In [28]:
in_progress_dataset_group_arns = [ dataset_group_without_desc_arn, dataset_group_with_desc_arn ]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for dataset_group_arn in in_progress_dataset_group_arns:
        describe_dataset_group_response = personalize.describe_dataset_group(
            datasetGroupArn = dataset_group_arn
        )
        status = describe_dataset_group_response["datasetGroup"]["status"]
        if status == "ACTIVE":
            print("Dataset group create succeeded for {}".format(dataset_group_arn))
            in_progress_dataset_group_arns.remove(dataset_group_arn)
        elif status == "CREATE FAILED":
            print("Create failed for {}".format(dataset_group_arn))
            in_progress_dataset_group_arns.remove(dataset_group_arn)

    if len(in_progress_dataset_group_arns) <= 0:
        break
    else:
        print("At least one dataset group create is still in progress")
                
    time.sleep(10)

At least one dataset group create is still in progress
Dataset group create succeeded for arn:aws:personalize:us-east-1:224124347618:dataset-group/amazon-pantry-without-desc
At least one dataset group create is still in progress
Dataset group create succeeded for arn:aws:personalize:us-east-1:224124347618:dataset-group/amazon-pantry-with-desc


### Crear conjuntos de datos y esquemas de conjuntos de datos de interacciones

Dado que el conjunto de datos de interacciones será el mismo para ambos grupos de conjunto de datos, crearemos un solo esquema para el tipo de conjunto de datos de interacciones y lo compartiremos con ambos grupos de conjunto de datos. Esto es posible, ya que los esquemas son globales a la cuenta de AWS y no específicos de un grupo de conjunto de datos.

In [29]:
interactions_schema = schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        },
        {
            "name": "EVENT_VALUE",
            "type": "float"
        },
        {
            "name": "EVENT_TYPE",
            "type": "string"
        }
    ],
    "version": "1.0"
}
            
create_schema_response = personalize.create_schema(
    name = "amazon-pantry-interactions",
    schema = json.dumps(interactions_schema)
)

interaction_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:224124347618:schema/amazon-pantry-interactions",
  "ResponseMetadata": {
    "RequestId": "6be5e019-0c8e-487b-a158-6f089f9e79ca",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:08:40 GMT",
      "x-amzn-requestid": "6be5e019-0c8e-487b-a158-6f089f9e79ca",
      "content-length": "92",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


A continuación, crearemos un conjunto de datos de interacciones en ambos grupos de conjunto de datos y especificaremos el esquema que acabamos de crear.

In [30]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "amazon-pantry-without-desc-ints",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_without_desc_arn,
    schemaArn = interaction_schema_arn
)

interactions_dataset_without_desc_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:224124347618:dataset/amazon-pantry-without-desc/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "ea1020bb-a7fa-4a64-a29e-f07e8e0f9ae9",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:08:41 GMT",
      "x-amzn-requestid": "ea1020bb-a7fa-4a64-a29e-f07e8e0f9ae9",
      "content-length": "107",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [31]:
create_dataset_response = personalize.create_dataset(
    name = "amazon-pantry-with-desc-ints",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_with_desc_arn,
    schemaArn = interaction_schema_arn
)

interactions_dataset_with_desc_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:224124347618:dataset/amazon-pantry-with-desc/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "614257cc-6344-408e-8e15-89d4f81ae927",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:08:41 GMT",
      "x-amzn-requestid": "614257cc-6344-408e-8e15-89d4f81ae927",
      "content-length": "104",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Preparar archivo CSV de interacciones en S3

Antes de poder cargar los CSV de interacciones que creamos antes en los conjuntos de datos de Personalize que acabamos de crear, necesitamos preparar el CSV en un bucket de S3.

Creemos un bucket de S3 y copiemos el archivo CSV de interacciones en el bucket.

In [32]:
# Determine the current S3 region where this notebook is being hosted in SageMaker.
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print(region)

us-east-1


In [33]:
s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
bucket_name = account_id + "-" + region + "-" + "amazon-pantry-personalize-text"
print(bucket_name)
if region == "us-east-1":
    s3.create_bucket(Bucket=bucket_name)
else:
    s3.create_bucket(
        Bucket=bucket_name,
        CreateBucketConfiguration={'LocationConstraint': region}
    )

224124347618-us-east-1-amazon-pantry-personalize-text


#### Cargar el CSV de interacciones en S3

In [34]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(interactions_filename).upload_file(interactions_filename)

### Crear política de bucket de S3 y rol de IAM

Antes de poder enviar un trabajo de importación de conjunto de datos a Personalize, debemos crear una política de bucket y un rol de IAM que otorgará a Personalize acceso a nuestro bucket.

In [35]:
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:*Object",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

{'ResponseMetadata': {'RequestId': 'SBN10P9R7H7ST5RK',
  'HostId': 'DD8fYEx27yBq6/rB7o9lMvkdCLOHOewN05NSq73g30jeFBdouLj5D+fWSnIZHvDuAKdCKEo7w3k=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'DD8fYEx27yBq6/rB7o9lMvkdCLOHOewN05NSq73g30jeFBdouLj5D+fWSnIZHvDuAKdCKEo7w3k=',
   'x-amz-request-id': 'SBN10P9R7H7ST5RK',
   'date': 'Tue, 13 Jul 2021 22:10:59 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [36]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleAmazonPantry"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(20) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

arn:aws:iam::224124347618:role/PersonalizeRoleAmazonPantry


### Importación de conjuntos de datos de interacciones para cada grupo de conjunto de datos

Ahora, estamos listos para importar el CSV de interacciones preparado en nuestro bucket de S3 a los conjuntos de datos de Personalize que creamos en cada grupo de conjunto de datos. Enviaremos ambos trabajos de importación y esperaremos a que se completen.

In [37]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "amazon-pantry-without-desc-ints-import",
    datasetArn = interactions_dataset_without_desc_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_filename)
    },
    roleArn = role_arn
)

dataset_import_job_without_ints_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:224124347618:dataset-import-job/amazon-pantry-without-desc-ints-import",
  "ResponseMetadata": {
    "RequestId": "84c4d71d-fe71-4ee7-bccf-4f8ed8b1e549",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:12:08 GMT",
      "x-amzn-requestid": "84c4d71d-fe71-4ee7-bccf-4f8ed8b1e549",
      "content-length": "126",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [38]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "amazon-pantry-with-desc-ints-import",
    datasetArn = interactions_dataset_with_desc_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_filename)
    },
    roleArn = role_arn
)

dataset_import_job_with_ints_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:224124347618:dataset-import-job/amazon-pantry-with-desc-ints-import",
  "ResponseMetadata": {
    "RequestId": "eae39716-264b-48e8-bfb4-93f569c7a904",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:12:09 GMT",
      "x-amzn-requestid": "eae39716-264b-48e8-bfb4-93f569c7a904",
      "content-length": "123",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Espere a que el trabajo de importación del conjunto de datos de interacciones se complete

La celda siguiente esperará a que ambos trabajos de importación se completen.

In [39]:
%%time

in_progress_import_arns = [ dataset_import_job_without_ints_arn, dataset_import_job_with_ints_arn ]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for import_arn in in_progress_import_arns:
        describe_dataset_import_job_response = personalize.describe_dataset_import_job(
            datasetImportJobArn = import_arn
        )
        status = describe_dataset_import_job_response["datasetImportJob"]['status']
        if status == "ACTIVE":
            print("Dataset import succeeded for {}".format(import_arn))
            in_progress_import_arns.remove(import_arn)
        elif status == "CREATE FAILED":
            print("Create failed for {}".format(import_arn))
            in_progress_import_arns.remove(import_arn)

    if len(in_progress_import_arns) <= 0:
        break
    else:
        print("At least one dataset import job is still in progress")
                
    time.sleep(60)

At least one dataset import job is still in progress
At least one dataset import job is still in progress
At least one dataset import job is still in progress
At least one dataset import job is still in progress
Dataset import succeeded for arn:aws:personalize:us-east-1:224124347618:dataset-import-job/amazon-pantry-without-desc-ints-import
At least one dataset import job is still in progress
Dataset import succeeded for arn:aws:personalize:us-east-1:224124347618:dataset-import-job/amazon-pantry-with-desc-ints-import
CPU times: user 42.3 ms, sys: 6.1 ms, total: 48.4 ms
Wall time: 5min


### Crear conjuntos de datos y esquema de conjuntos de datos de elementos

A continuación, repetiremos el proceso para los conjuntos de datos de elementos. Sin embargo, esta vez crearemos dos esquemas, ya que un conjunto de datos de elementos contiene la columna de descripción y el otro no. Comenzaremos con el esquema que no incluye la descripción.

In [40]:
item_without_desc_schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "BRAND",
            "type": [ "null", "string" ],
            "categorical": True
        },{
            "name": "PRICE",
            "type": [ "null", "float" ],
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "amazon-pantry-item-without-desc-schema",
    schema = json.dumps(item_without_desc_schema)
)

item_without_desc_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:224124347618:schema/amazon-pantry-item-without-desc-schema",
  "ResponseMetadata": {
    "RequestId": "11a18b05-189b-4485-ba8b-58083e784321",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:17:16 GMT",
      "x-amzn-requestid": "11a18b05-189b-4485-ba8b-58083e784321",
      "content-length": "104",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


A continuación, crearemos el esquema que incluye la descripción. Asegúrese de prestar atención al atributo `"textual": True` en el campo `DESCRIPTION`. De esta manera se diferencian los campos de texto no estructurados de los campos categóricos y de cadena. Sin este atributo, Personalize no aplicará técnicas de procesamiento de lenguaje natural para extraer características desde este texto.

In [41]:
item_with_desc_schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "BRAND",
            "type": [ "null", "string" ],
            "categorical": True
        },{
            "name": "PRICE",
            "type": [ "null", "float" ],
        },{
            "name": "DESCRIPTION",
            "type": [ "null", "string" ],
            "textual": True
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "amazon-pantry-item-with-desc-schema",
    schema = json.dumps(item_with_desc_schema)
)

item_with_desc_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:224124347618:schema/amazon-pantry-item-with-desc-schema",
  "ResponseMetadata": {
    "RequestId": "05c60e80-1d7c-45f1-a881-d08af62a8432",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:17:16 GMT",
      "x-amzn-requestid": "05c60e80-1d7c-45f1-a881-d08af62a8432",
      "content-length": "101",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


A continuación, crearemos conjuntos de datos de Personalize en cada grupo de conjunto de datos. Tenga precaución al momento de especificar el esquema de ARN apropiado para cada conjunto de datos.

In [42]:
dataset_type = "ITEMS"
create_dataset_response = personalize.create_dataset(
    name = "amazon-pantry-without-desc-items",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_without_desc_arn,
    schemaArn = item_without_desc_schema_arn
)

items_dataset_without_desc_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:224124347618:dataset/amazon-pantry-without-desc/ITEMS",
  "ResponseMetadata": {
    "RequestId": "f9b87f6c-22c8-42a7-8a3a-8c203300e3eb",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:18:13 GMT",
      "x-amzn-requestid": "f9b87f6c-22c8-42a7-8a3a-8c203300e3eb",
      "content-length": "100",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [43]:
create_dataset_response = personalize.create_dataset(
    name = "amazon-pantry-with-desc-items",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_with_desc_arn,
    schemaArn = item_with_desc_schema_arn
)

items_dataset_with_desc_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:224124347618:dataset/amazon-pantry-with-desc/ITEMS",
  "ResponseMetadata": {
    "RequestId": "cbba4d2a-17c9-4a9b-a6a3-a2df934e2de1",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:18:16 GMT",
      "x-amzn-requestid": "cbba4d2a-17c9-4a9b-a6a3-a2df934e2de1",
      "content-length": "97",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Preparación del CSV de elementos en S3

Luego, copiaremos nuestros dos archivos CSV de elementos al mismo bucket de S3 que creamos más arriba.

In [44]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(items_without_desc_filename).upload_file(items_without_desc_filename)

In [45]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(items_with_desc_filename).upload_file(items_with_desc_filename)

### Importar conjuntos de datos de elementos para cada grupo de conjuntos de datos

Ahora que la política del bucket de S3 y el rol de IAM están configurados, podemos enviar los dos trabajos de importación de conjunto de datos para importar los CSV de elementos.

In [46]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "amazon-pantry-without-desc-items-import",
    datasetArn = items_dataset_without_desc_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, items_without_desc_filename)
    },
    roleArn = role_arn
)

dataset_import_job_without_items_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:224124347618:dataset-import-job/amazon-pantry-without-desc-items-import",
  "ResponseMetadata": {
    "RequestId": "c6ea207b-b8eb-4565-8ce2-6eb44e0fa18f",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:18:37 GMT",
      "x-amzn-requestid": "c6ea207b-b8eb-4565-8ce2-6eb44e0fa18f",
      "content-length": "127",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [47]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "amazon-pantry-with-desc-items-import",
    datasetArn = items_dataset_with_desc_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, items_with_desc_filename)
    },
    roleArn = role_arn
)

dataset_import_job_with_items_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:224124347618:dataset-import-job/amazon-pantry-with-desc-items-import",
  "ResponseMetadata": {
    "RequestId": "cf95206f-1527-4247-a618-6c8c832fa05f",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:18:38 GMT",
      "x-amzn-requestid": "cf95206f-1527-4247-a618-6c8c832fa05f",
      "content-length": "124",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Espere a que los trabajos de importación de elementos se completen

La lógica siguiente esperará hasta que ambos conjuntos de datos de elementos estén importados por completo a cada grupo de conjunto de datos.

In [48]:
%%time

in_progress_import_arns = [ dataset_import_job_without_items_arn, dataset_import_job_with_items_arn ]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for import_arn in in_progress_import_arns:
        describe_dataset_import_job_response = personalize.describe_dataset_import_job(
            datasetImportJobArn = import_arn
        )
        status = describe_dataset_import_job_response["datasetImportJob"]['status']
        if status == "ACTIVE":
            print("Dataset import succeeded for {}".format(import_arn))
            in_progress_import_arns.remove(import_arn)
        elif status == "CREATE FAILED":
            print("Create failed for {}".format(import_arn))
            in_progress_import_arns.remove(import_arn)

    if len(in_progress_import_arns) <= 0:
        break
    else:
        print("At least one dataset import job is still in progress")
                
    time.sleep(60)

At least one dataset import job is still in progress
At least one dataset import job is still in progress
At least one dataset import job is still in progress
At least one dataset import job is still in progress
Dataset import succeeded for arn:aws:personalize:us-east-1:224124347618:dataset-import-job/amazon-pantry-with-desc-items-import
At least one dataset import job is still in progress
At least one dataset import job is still in progress
At least one dataset import job is still in progress
Dataset import succeeded for arn:aws:personalize:us-east-1:224124347618:dataset-import-job/amazon-pantry-without-desc-items-import
CPU times: user 57.6 ms, sys: 5.06 ms, total: 62.7 ms
Wall time: 7min


## Crear soluciones y versiones de soluciones

Ahora que importamos las interacciones y los conjuntos de datos de elementos a cada grupo de conjunto de datos, crearemos soluciones y versiones de soluciones mediante la receta de personalización de usuario para los datos en cada grupo de conjunto de datos.

Primero, enumeremos las recetas de Personalize disponibles.

In [49]:
personalize.list_recipes()

{'recipes': [{'name': 'aws-hrnn',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2021, 2, 6, 19, 6, 40, 447000, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-coldstart',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-coldstart',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2021, 2, 6, 19, 6, 40, 447000, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-metadata',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-metadata',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2021, 2, 6, 19, 6, 40, 447000, tzinfo=tzlocal())},
  {'name': 'aws-personalized-ranking',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-personalized-ranking',
   'stat

Para este cuaderno, utilizaremos la receta de user-personalization, ya que es una de las recetas que utiliza metadatos de elementos. Esta receta es compatible con el caso de uso de personalización canónica, donde, dado un usuario, quiere que Personalize le recomiende elementos en los que este usuario estaría interesado. 

In [50]:
user_personalization_recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization"

En primer lugar, crearemos una solución y una versión de solución en el grupo de conjunto de datos que no incluyen descripciones de los elementos.

In [51]:
user_personalization_create_solution_response = personalize.create_solution(
    name = "amazon-pantry-without-desc-userpersonalization",
    datasetGroupArn = dataset_group_without_desc_arn,
    recipeArn = user_personalization_recipe_arn
)

user_personalization_without_desc_solution_arn = user_personalization_create_solution_response['solutionArn']

In [52]:
print(user_personalization_without_desc_solution_arn)

arn:aws:personalize:us-east-1:224124347618:solution/amazon-pantry-without-desc-userpersonalization


In [53]:
user_personalization_solution_version_response = personalize.create_solution_version(
    solutionArn = user_personalization_without_desc_solution_arn
)

In [54]:
user_personalization_without_solution_version_arn = user_personalization_solution_version_response['solutionVersionArn']
print(json.dumps(user_personalization_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:224124347618:solution/amazon-pantry-without-desc-userpersonalization/0b76212f",
  "ResponseMetadata": {
    "RequestId": "018b3fcb-10d5-4290-a17a-3970723abacd",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:26:14 GMT",
      "x-amzn-requestid": "018b3fcb-10d5-4290-a17a-3970723abacd",
      "content-length": "132",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


A continuación, crearemos una solución y una versión de solución en el grupo de conjunto de datos que incluyen descripciones de los elementos.

In [55]:
user_personalization_create_solution_response = personalize.create_solution(
    name = "amazon-pantry-with-desc-userpersonalization",
    datasetGroupArn = dataset_group_with_desc_arn,
    recipeArn = user_personalization_recipe_arn
)

user_personalization_with_desc_solution_arn = user_personalization_create_solution_response['solutionArn']

In [56]:
print(user_personalization_with_desc_solution_arn)

arn:aws:personalize:us-east-1:224124347618:solution/amazon-pantry-with-desc-userpersonalization


In [57]:
user_personalization_solution_version_response = personalize.create_solution_version(
    solutionArn = user_personalization_with_desc_solution_arn
)

In [58]:
user_personalization_with_solution_version_arn = user_personalization_solution_version_response['solutionVersionArn']
print(json.dumps(user_personalization_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:224124347618:solution/amazon-pantry-with-desc-userpersonalization/f178990f",
  "ResponseMetadata": {
    "RequestId": "f630b862-6fa9-4eb7-a0d2-71d0b9637e80",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:26:29 GMT",
      "x-amzn-requestid": "f630b862-6fa9-4eb7-a0d2-71d0b9637e80",
      "content-length": "129",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Esperar a que se activen las versiones de solución

Por último, esperaremos a que se terminen de crear las versiones de la solución. En este paso, Personalize entrena modelos de machine learning basados en los conjuntos de datos y la receta seleccionada. También, Personalize dividirá los conjuntos de datos de interacciones en porciones de prueba y evaluación para poder contrastar la calidad de las recomendaciones con las del modelo de prueba mediante datos retenidos.

Notará que la versión de la solución en el grupo de conjunto de datos que incluye los datos de descripción tardará más tiempo en probarse que el grupo sin descripciones.

In [59]:
%%time

in_progress_solution_versions = [
    user_personalization_without_solution_version_arn,
    user_personalization_with_solution_version_arn
]

max_time = time.time() + 10*60*60 # 10 hours
while time.time() < max_time:
    for solution_version_arn in in_progress_solution_versions:
        version_response = personalize.describe_solution_version(
            solutionVersionArn = solution_version_arn
        )
        status = version_response["solutionVersion"]["status"]
        
        if status == "ACTIVE":
            print("Build succeeded for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
        elif status == "CREATE FAILED":
            print("Build failed for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
    
    if len(in_progress_solution_versions) <= 0:
        break
    else:
        print("At least one solution build is still in progress")
        
    time.sleep(60)

At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solutio

En general, cuando agregue metadatos no estructurados basados en texto se incrementará el tiempo de prueba. En nuestro caso, verá que la versión de la solución que realizó la prueba en el conjunto de datos con descripciones de producto demoró alrededor de 15 minutos más que la versión de la solución que realizó la prueba en el conjunto de datos sin las descripciones. Esta diferencia va a variar según la composición y los valores de datos de los conjuntos de datos.

Revisemos las horas de entrenamiento para cada versión de la solución y comparémoslas.

In [60]:
response = personalize.describe_solution_version(solutionVersionArn = user_personalization_without_solution_version_arn)
training_hours_without_desc = response['solutionVersion']['trainingHours']

response = personalize.describe_solution_version(solutionVersionArn = user_personalization_with_solution_version_arn)
training_hours_with_desc = response['solutionVersion']['trainingHours']
training_diff = (training_hours_with_desc - training_hours_without_desc) / training_hours_without_desc

print(f"Training hours without description: {training_hours_without_desc}")
print(f"Training hours with description: {training_hours_with_desc}")

print("Difference of {:.2%}".format(training_diff))

Training hours without description: 4.199
Training hours with description: 5.346
Difference of 27.32%


Las horas de entrenamiento que se utilizaron para calcular los costos fueron un 27 % más altas cuando se utilizaron para entrenar con la columna de descripciones. 

El tiempo real transcurrido y las horas de entrenamiento variarán dependiendo en el tamaño de los conjuntos de datos. Pero, esta información le ayudará a evaluar la compensación al momento de considerar agregar texto no estructurado a sus conjuntos de datos.

### Revisión de métricas sin conexión

Ahora que las versiones de la solución terminaron de crearse, revisemos y comparemos las métricas sin conexión para cada versión de la solución para ver cómo la inclusión de texto sin estructura impactó estas métricas.

In [61]:
metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = user_personalization_without_solution_version_arn
)

print(json.dumps(metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:224124347618:solution/amazon-pantry-without-desc-userpersonalization/0b76212f",
  "metrics": {
    "coverage": 0.0914,
    "mean_reciprocal_rank_at_25": 0.0268,
    "normalized_discounted_cumulative_gain_at_10": 0.0376,
    "normalized_discounted_cumulative_gain_at_25": 0.0464,
    "normalized_discounted_cumulative_gain_at_5": 0.0309,
    "precision_at_10": 0.0058,
    "precision_at_25": 0.0037,
    "precision_at_5": 0.0076
  },
  "ResponseMetadata": {
    "RequestId": "8c61339a-f929-47e0-81f0-a9660ebd589f",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 23:16:07 GMT",
      "x-amzn-requestid": "8c61339a-f929-47e0-81f0-a9660ebd589f",
      "content-length": "430",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Vamos a guardarlas en un diccionario para que sea más sencillo comparar las métricas entre ambas versiones de solución.

In [62]:
metrics = {
    'Coverage': [ metrics_response['metrics']['coverage'] ],
    'MRR-25': [ metrics_response['metrics']['mean_reciprocal_rank_at_25'] ],
    'NDCG-5': [ metrics_response['metrics']['normalized_discounted_cumulative_gain_at_5'] ],
    'NDCG-10': [ metrics_response['metrics']['normalized_discounted_cumulative_gain_at_10'] ],
    'NDCG-25': [ metrics_response['metrics']['normalized_discounted_cumulative_gain_at_25'] ],    
    'Precision-5': [ metrics_response['metrics']['precision_at_5'] ],
    'Precision-10': [ metrics_response['metrics']['precision_at_10'] ],
    'Precision-25': [ metrics_response['metrics']['precision_at_25'] ],    
}

A continuación, guarde las métricas para la versión de la solución que incluye la descripción.

In [63]:
metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = user_personalization_with_solution_version_arn
)

print(json.dumps(metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:224124347618:solution/amazon-pantry-with-desc-userpersonalization/f178990f",
  "metrics": {
    "coverage": 0.1323,
    "mean_reciprocal_rank_at_25": 0.0367,
    "normalized_discounted_cumulative_gain_at_10": 0.049,
    "normalized_discounted_cumulative_gain_at_25": 0.0591,
    "normalized_discounted_cumulative_gain_at_5": 0.0425,
    "precision_at_10": 0.0071,
    "precision_at_25": 0.0045,
    "precision_at_5": 0.0104
  },
  "ResponseMetadata": {
    "RequestId": "b54df693-4378-4194-96c7-cec3a9d934cf",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 23:16:14 GMT",
      "x-amzn-requestid": "b54df693-4378-4194-96c7-cec3a9d934cf",
      "content-length": "426",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [64]:
metrics['Coverage'].append(metrics_response['metrics']['coverage'])
metrics['MRR-25'].append(metrics_response['metrics']['mean_reciprocal_rank_at_25'])
metrics['NDCG-5'].append(metrics_response['metrics']['normalized_discounted_cumulative_gain_at_5'])
metrics['NDCG-10'].append(metrics_response['metrics']['normalized_discounted_cumulative_gain_at_10'])
metrics['NDCG-25'].append(metrics_response['metrics']['normalized_discounted_cumulative_gain_at_25'])
metrics['Precision-5'].append(metrics_response['metrics']['precision_at_5'])
metrics['Precision-10'].append(metrics_response['metrics']['precision_at_10'])
metrics['Precision-25'].append(metrics_response['metrics']['precision_at_25'])

Calcule las diferencias en el porcentaje en cada métrica, con texto y sin él, y muestre los resultados.

In [65]:
for key in metrics:
    metrics[key].append("{:.2%}".format((metrics[key][1] - metrics[key][0])/metrics[key][0]))

metrics_df = pd.DataFrame.from_dict(metrics,orient='index',columns=['Without Text', 'With Text', '% Change'])
metrics_df

,Without Text,With Text,% Change
Coverage,0.0914,0.1323,44.75%
MRR-25,0.0268,0.0367,36.94%
NDCG-5,0.0309,0.0425,37.54%
NDCG-10,0.0376,0.0490,30.32%
NDCG-25,0.0464,0.0591,27.37%
Precision-5,0.0076,0.0104,36.84%
Precision-10,0.0058,0.0071,22.41%
Precision-25,0.0037,0.0045,21.62%


Estas métricas muestran claramente que las recomendaciones desde la versión de solución que incluye las descripciones de los elementos fueron mejores de manera significativa en todos los aspectos. Para más conjuntos de datos de interacciones escasas, donde los usuarios y los elementos tienen menos interacciones, se beneficiarán más con la adición de texto que con conjuntos de datos que ya tienen un número más alto de interacciones por elemento y usuario.

## Limpiar

Los recursos de Personalize que se creen con este cuaderno pueden eliminarse desde la página de servicio de Personalize en la consola de AWS. 

De manera alternativa, el siguiente script se puede ejecutar de manera local para eliminar todos los recursos en cada grupo de conjunto de datos.

https://gist.github.com/james-jory/62ddddf2f9180b77dd2a42e645b9d3b0

Además, el rol de IAM y el bucket de S3 pueden eliminarse desde las páginas de servicio de IAM y S3, respectivamente.